# How to find data in an Orthanc server

This notebook assume that you have `pyorthanc` installed and an Orthanc server run on your machine at `http://localhost:8042`.

The easiest solution to install it is with docker, using: `docker run -p 8042:8042 -p 4242:4242 orthancteam/orthanc`


In [1]:
import pyorthanc

In [2]:
# Creating the Orthanc client
orthanc = pyorthanc.Orthanc(
    url='http://localhost:8042',  # URL of you Orthanc server
    username='orthanc',  # Using the default username
    password='orthanc',  # Using the default password
)

### Upload data
If you Orthanc server doesn't have data, you can upload a few example from the `pydicom` library.

In [3]:
from pydicom.data import get_testdata_file

small_ct_path = get_testdata_file('CT_small.dcm')
pyorthanc.upload(orthanc, small_ct_path)

{'ID': 'f689ddd2-662f8fe1-8b18180d-ec2a2cee-937917af',
 'ParentPatient': 'fa558bce-587a86d3-ad0da9b3-9d043d9d-4f5c5718',
 'ParentSeries': '93034833-163e42c3-bc9a428b-194620cf-2c5799e5',
 'ParentStudy': '8a8cf898-ca27c490-d0c7058c-929d0581-2bbf104d',
 'Path': '/instances/f689ddd2-662f8fe1-8b18180d-ec2a2cee-937917af',
 'Status': 'Success'}

## Make queries to find you data

Using `find_patients` allow you to find your patients efficiently, even on large datasets.

In [10]:
# "patients" is a list of "pyorthanc.Patient" objects that fit the query
patients = pyorthanc.find_patients(
    orthanc,
    query={'PatientID': '*', 'PatientName': '*'}  # Here you can put the Main DICOM tags supported by Orthanc
)
print('Patients that fit the query:', patients)

Patients that fit the query: [Patient(fa558bce-587a86d3-ad0da9b3-9d043d9d-4f5c5718)]


In [9]:
# Studies, Series and instance data are also available through the pyorthanc.Patient object.
for patient in patients:
    print('Name:', patient.name)
    for study in patient.studies:
        print('Study date:', study.date)
        for series in study.series:
            ...

Name: CompressedSamples^CT1
Study date: 2004-01-19 07:27:30


The other functions `find_studies`, `find_series` and `find_instances` are also useful to find your desired data!

In [11]:
series = pyorthanc.find_series(
    orthanc,
    query={'PatientID': '*', 'Modality': 'CT'}  # Here you can put the Main DICOM tags supported by Orthanc
)

print('Series that fit the query:', series)

Series that fit the query: [Series(93034833-163e42c3-bc9a428b-194620cf-2c5799e5)]


Note that it is possible to retrieve the DICOM instance Dataset from the `pyorthanc.Instance` object.

In [13]:
instance = series[0].instances[0]

# Having the pydicom.Dataset object make it easier to do complex data operation.
ds = instance.get_pydicom()
print(ds.PatientID)
print(ds.PatientName)
print(ds.StudyDate)
print(ds.Modality)
...

1CT1
CompressedSamples^CT1
20040119
CT


Ellipsis